# Advanced Multi-Agent Orchestration with A2A and Llama Stack

This notebook demonstrates how to construct and orchestrate a multi-agent system using the Agent-to-Agent (A2A) communication protocol. We will build individual agents using Llama Stack and then enable them to collaborate on complex tasks by exposing their functionalities via A2A servers. 

This demo focuses on an orchestration pattern where a planner agent determines which specialized agent (skill) to call, and a composer agent formats the final response.

## Overview

This notebook covers the following steps:

1.  **Setting up the Llama Stack Environment**: Initializing the Llama Stack client and configuring model parameters.

2.  **Defining Llama Stack Agents**: Creating three distinct Llama Stack agents:

    * `Planner Agent`: Responsible for interpreting user queries and creating a plan to use other agents' skills.

    * `Custom Tool Agent`: Equipped with tools for random number generation and date retrieval.

    * `Composer Agent`: Skilled at generating human-friendly text from structured data.

3.  **Serving Llama Stack Agents via A2A**: Exposing each Llama Stack agent over an individual A2A server, making their `AgentCard` skills accessible via the A2A protocol.

4.  **Orchestrating the A2A Agents**: Setting up an `AgentManager` to manage communication with the A2A-enabled agents and implementing an `orchestrate` function to coordinate them.

5.  **Running the Orchestration**: Launching the multi-agent system to answer user queries, leveraging the planner to select tools/skills and the composer to generate a final, human-readable response.


## Prerequisites

Before starting, ensure you have the following:
- `python_requires >= 3.13`

- Followed the instructions in the [Setup Guide](../../rag_agentic/notebooks/Level0_getting_started_with_Llama_Stack.ipynb) notebook.

## Additional environment variables
This demo requires the following environment variables in addition to those defined in the [Setup Guide](../../rag_agentic/notebooks/Level0_getting_started_with_Llama_Stack.ipynb):

- `PLANNER_AGENT_LOCAL_PORT`: The port for the A2A agent responsible for planning (e.g. 10020).

- `CUSTOM_TOOL_AGENT_LOCAL_PORT`: The port for the A2A agent with custom tool capabilities (e.g. 10021).

- `COMPOSER_AGENT_LOCAL_PORT`: The port for the A2A agent responsible for composing final answers (e.g. 10022).

## 1. Setting Up this Notebook
To provide A2A communication capabilities, we will use the [sample implementation by Google](https://github.com/google/A2A/tree/main/samples/python). Please make sure that the content of the referenced directory is available on your Python path. This can be done, for example, by running the following command:

In [1]:
! git clone https://github.com/google-a2a/a2a-samples.git
! pip install -r "../requirements.txt"

fatal: destination path 'a2a-samples' already exists and is not an empty directory.
  Using cached annotated_types-0.7.0-py3-none-any.whl.metadata (15 kB)
  Using cached anyio-4.9.0-py3-none-any.whl.metadata (4.7 kB)
  Using cached certifi-2025.1.31-py3-none-any.whl.metadata (2.5 kB)
  Using cached cffi-1.17.1-cp313-cp313-macosx_11_0_arm64.whl.metadata (1.5 kB)
  Using cached charset_normalizer-3.4.2-cp313-cp313-macosx_10_13_universal2.whl.metadata (35 kB)
  Using cached click-8.1.8-py3-none-any.whl.metadata (2.3 kB)
  Using cached cryptography-45.0.3-cp311-abi3-macosx_10_9_universal2.whl.metadata (5.7 kB)
  Using cached distro-1.9.0-py3-none-any.whl.metadata (6.8 kB)
  Using cached dotenv-0.9.9-py2.py3-none-any.whl.metadata (279 bytes)
  Using cached fire-0.7.0-py3-none-any.whl
  Using cached h11-0.16.0-py3-none-any.whl.metadata (8.3 kB)
  Using cached httpcore-1.0.9-py3-none-any.whl.metadata (21 kB)
  Using cached httpx-0.28.1-py3-none-any.whl.metadata (7.1 kB)
  Using cached httpx_s

Now, we will add the paths to the A2A library and our own tools to `sys.path`.

In [2]:
import sys
# the path of the A2A library
sys.path.append('./a2a-samples/samples/python')
# the path to our own utils
sys.path.append('../..')

We will now proceed with the necessary imports.

In [3]:
from common.server import A2AServer
from common.types import AgentCard, AgentSkill, AgentCapabilities
from common.client import A2AClient, A2ACardResolver
from common.utils.push_notification_auth import PushNotificationReceiverAuth
from hosts.cli.push_notification_listener import PushNotificationListener

from a2a_llama_stack.A2ATool import A2ATool
from a2a_llama_stack.task_manager import AgentTaskManager

# for asynchronously serving the A2A agent
import threading


import json
import urllib.parse
from uuid import uuid4
from typing import Any, Dict, List, Tuple

Next, we will initialize our environment as described in detail in our ["Getting Started" notebook](../../rag_agentic/notebooks/Level0_getting_started_with_Llama_Stack.ipynb). Please refer to it for additional explanations.

In [4]:
# for accessing the environment variables
import os
from dotenv import load_dotenv
load_dotenv()

# for communication with Llama Stack
from llama_stack_client import LlamaStackClient

# agent related imports
import uuid
from llama_stack_client import Agent
# from llama_stack_client.lib.agents.event_logger import EventLogger


base_url = os.getenv("REMOTE_BASE_URL")


# Tavily search API key is required for some of our demos and must be provided to the client upon initialization.
# We will cover it in the agentic demos that use the respective tool. Please ignore this parameter for all other demos.
tavily_search_api_key = os.getenv("TAVILY_SEARCH_API_KEY")
if tavily_search_api_key is None:
    provider_data = None
else:
    provider_data = {"tavily_search_api_key": tavily_search_api_key}


client = LlamaStackClient(
    base_url=base_url,
    provider_data=provider_data
)
    
print(f"Connected to Llama Stack server")

# model_id for the model you wish to use that is configured with the Llama Stack server
model_id = os.getenv("INFERENCE_MODEL_ID")

temperature = float(os.getenv("TEMPERATURE", 0.0))
if temperature > 0.0:
    top_p = float(os.getenv("TOP_P", 0.95))
    strategy = {"type": "top_p", "temperature": temperature, "top_p": top_p}
else:
    strategy = {"type": "greedy"}

max_tokens = int(os.getenv("MAX_TOKENS", 4096))

# sampling_params will later be used to pass the parameters to Llama Stack Agents/Inference APIs
sampling_params = {
    "strategy": strategy,
    "max_tokens": max_tokens,
}

stream_env = os.getenv("STREAM", "False")
# the Boolean 'stream' parameter will later be passed to Llama Stack Agents/Inference APIs
# any value non equal to 'False' will be considered as 'True'
stream = (stream_env != "False")

print(f"Inference Parameters:\n\tModel: {model_id}\n\tSampling Parameters: {sampling_params}\n\tstream: {stream}")

Connected to Llama Stack server
Inference Parameters:
	Model: llama3.1:8b-instruct-fp16
	Sampling Parameters: {'strategy': {'type': 'greedy'}, 'max_tokens': 4096}
	stream: False


## 2. Setting Up and Serving A2A Agents
Now, we will define the core Llama Stack agents that will form our multi-agent system. These agents are created using the Llama Stack `Agent` class. Later, we will expose their functionalities via A2A servers.

We will initialize three distinct Llama Stack agents:

1. **Planner Agent**: an agent that acts as an orchestrator, determining which skills (other agents) are needed to answer a user's query.

2. **Custom Tool Agent**: an agent equipped with tools to generate random numbers and provide the current date.

3. **Composer Agent**: an agent skilled at writing human-friendly text based on provided information.

#### 2.1. Planner Agent
The Planner Agent is responsible for understanding the user's query and determining which skills (exposed by other agents) are needed to fulfill the request. It outputs a plan, typically a list of skill IDs to be invoked.

In [5]:
planner_agent = Agent(
    client,
    model=model_id,
    instructions=("You are an orchestration assistant. Ensure you count correctly the number of skills needed."),
    sampling_params=sampling_params,
    tools=[],
    max_infer_iters=10,
)

#### 2.2. Custom Tool Agent
First, we define the Python functions that will serve as custom tools for our second agent.

In [6]:
import random
from datetime import datetime


def random_number_tool() -> int:
    """
    Generate a random integer between 1 and 100.
    """
    print("\n\nGenerating a random number...\n\n")
    return random.randint(1, 100)


def date_tool() -> str:
    """
    Return today's date in YYYY-MM-DD format.
    """
    return datetime.utcnow().date().isoformat()

Next, initialize the Llama Stack agent, providing it with these tools and instructions on how to use them.

In [7]:
custom_tool_agent = Agent(
    client,
    model=model_id,
    instructions=(
            "You have access to two tools:\n"
            "- random_number_tool: generates one random integer between 1 and 100\n"
            "- date_tool: returns today's date in YYYY-MM-DD format\n"
            "Always use the appropriate tool to answer user queries."
        ),    
    sampling_params=sampling_params,
    tools=[random_number_tool, date_tool],
    max_infer_iters=3,
)

#### 2.3. Composer Agent
Finally, we initialize the Composer Agent. This agent's role is to take structured data (e.g. results from other tools or agents) and formulate a coherent, human-friendly response.

In [8]:
composer_agent = Agent(
    client,
    model=model_id,
    instructions=("You are skilled at writing human-friendly text based on the query and associated skills."),   
    sampling_params=sampling_params,
    tools=[],
    max_infer_iters=3,
)

## 3. Serving Llama Stack Agents via A2A
Now that we have our Llama Stack agents, we need to make their functionalities accessible via the A2A protocol. This involves:

- Creating an `AgentCard`: An object containing metadata about the agent, including its URL and exposed capabilities (`AgentSkill`).

- Wrapping the Llama Stack agent with an `AgentTaskManager`: An adapter that allows the A2A server to forward requests to the Llama Stack agent.

- Creating and launching an `A2AServer`: A REST API server that handles A2A protocol communication for this agent.

#### 3.1. Serving the Planner Agent
First, we serve the Planner Agent via its own A2A server. Its `AgentCard` will highlight its orchestration planning skill.

In [9]:
planner_agent_local_port = int(os.getenv("planner_agent_LOCAL_PORT", "10020"))
planner_agent_url = f"http://localhost:{planner_agent_local_port}"

agent_card = AgentCard(
    name="Orchestration Agent",
    description="Plans which tool to call for each user question",
    url=planner_agent_url,
    version="0.1.0",
    defaultInputModes=["text/plain"],
    defaultOutputModes=["text/plain"],
    capabilities=AgentCapabilities(
        streaming=False,
        pushNotifications=False,
        stateTransitionHistory=False,
        ),
    skills=[
        AgentSkill(
            id="orchestrate",
            name="Orchestration Planner",
            description="Plan user questions into JSON steps of {skill_id}",
            tags=["orchestration"],
            examples=["Plan: What's today's date and a random number?"],
            inputModes=["text/plain"],
            outputModes=["application/json"],
            ),
    ],
)
task_manager = AgentTaskManager(agent=planner_agent)
server = A2AServer(
    agent_card=agent_card,
    task_manager=task_manager,
    host='localhost',
    port=planner_agent_local_port
)
thread = threading.Thread(target=server.start, daemon=True)
thread.start()

INFO:     Started server process [85772]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
ERROR:    [Errno 48] error while attempting to bind on address ('::1', 10020, 0, 0): [errno 48] address already in use
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.


#### 3.2. Serving the Custom Tool Agent
We create an `AgentCard` for the Custom Tool Agent, detailing its skills (random number generation and date retrieval). Then, we wrap our `custom_tool_agent` (the Llama Stack Agent) in an `AgentTaskManager` and start an A2AServer for it.

In [ ]:
custom_tool_agent_local_port = int(os.getenv("CUSTOM_TOOL_AGENT_LOCAL_PORT", "10021"))
custom_tool_agent_url = f"http://localhost:{custom_tool_agent_local_port}"

agent_card = AgentCard(
    name="Custom Agent",
    description="Generates random numbers or retrieve today's dates",
    url=custom_tool_agent_url,
    version="0.1.0",
    defaultInputModes=["text/plain"],
    defaultOutputModes=["text/plain"],
    capabilities=AgentCapabilities(
        streaming=False,
        pushNotifications=False,
        stateTransitionHistory=False,
        ),
    skills=[
        AgentSkill(
            id="random_number_tool", 
            name="Random Number Generator",
            description="Generates a random number between 1 and 100",
            tags=["random"],
            examples=["Give me a random number between 1 and 100"],
            inputModes=["text/plain"],
            outputModes=["text/plain"],
            ),
        AgentSkill(
            id="date_tool",
            name="Date Provider",
            description="Returns today's date in YYYY-MM-DD format",
            tags=["date"],
            examples=["What's the date today?"],
            inputModes=["text/plain"],
            outputModes=["text/plain"],
            ),
    ],
)
task_manager = AgentTaskManager(agent=custom_tool_agent)
server = A2AServer(
    agent_card=agent_card,
    task_manager=task_manager,
    host='localhost',
    port=custom_tool_agent_local_port
)
thread = threading.Thread(target=server.start, daemon=True)
thread.start()

INFO:     Started server process [85772]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
ERROR:    [Errno 48] error while attempting to bind on address ('127.0.0.1', 10021): [errno 48] address already in use
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.


#### 3.3. Serving the Composer Agent
Similarly, we set up an A2A server for the Composer Agent. Its `AgentCard` will define its writing skill.

In [11]:
composer_agent_local_port = int(os.getenv("COMPOSER_AGENT_LOCAL_PORT", "10022"))
composer_agent_url = f"http://localhost:{composer_agent_local_port}"

agent_card = AgentCard(
    name="Writing Agent",
    description="Generate human-friendly text based on the query and associated skills",
    url=composer_agent_url,
    version="0.1.0",
    defaultInputModes=["text/plain"],
    defaultOutputModes=["text/plain"],
    capabilities=AgentCapabilities(
        streaming=False,
        pushNotifications=False,
        stateTransitionHistory=False,
        ),
    skills=[
        AgentSkill(
            id="writing_agent", 
            name="Writing Agent",
            description="Write human-friendly text based on the query and associated skills",
            tags=["writing"],
            examples=["Write human-friendly text based on the query and associated skills"],
            inputModes=["text/plain"],
            outputModes=["application/json"],
            ),
    ],
)
task_manager = AgentTaskManager(agent=composer_agent)
server = A2AServer(
    agent_card=agent_card,
    task_manager=task_manager,
    host='localhost',
    port=composer_agent_local_port
)
thread = threading.Thread(target=server.start, daemon=True)
thread.start()

INFO:     Started server process [85772]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
ERROR:    [Errno 48] error while attempting to bind on address ('::1', 10022, 0, 0): [errno 48] address already in use
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.


## 4. Orchestrating the A2A Agents
With all Llama Stack agents defined and served via A2A, we now set up the client-side logic to interact with them. This involves managing connections and coordinating the flow of information between the Planner Agent, Skill Executor Agents (Custom Tool Agent), and the Composer Agent.

#### 4.1. Agent Manager
The `AgentManager` class helps manage connections and agent cards for the orchestrator (Planner Agent) and the skill executor agents (Custom Tool Agent, Composer Agent). It simplifies accessing their A2A clients and metadata.

In [12]:
AgentInfo = Tuple[str, Any, A2AClient, str]

class AgentManager:
    def __init__(self, urls: List[str]):
        # first URL is your orchestrator…
        self.orchestrator: AgentInfo = self._make_agent_info(urls[0])
        # …the rest are skill agents, each keyed by skill.id
        self.skills: Dict[str, AgentInfo] = {
            skill.id: info
            for url in urls[1:]
            for info in (self._make_agent_info(url),)
            for skill in info[1].skills
        }

    @staticmethod
    def _make_agent_info(url: str) -> AgentInfo:
        card   = A2ACardResolver(url).get_agent_card()
        client = A2AClient(agent_card=card)
        session = uuid4().hex
        return url, card, client, session

#### 4.2. Orchestration Helper Functions
These asynchronous helper functions are used by the main orchestration logic to send tasks to the A2A agents and process their responses.


In [13]:
async def _send_payload(client, card, session, payload, streaming: bool) -> str:
    if not streaming:
        res = await client.send_task(payload)
        return res.result.status.message.parts[0].text.strip()

    text = ""
    async for ev in client.send_task_streaming(payload):
        part = ev.result.status.message.parts[0].text or ""
        print(part, end="", flush=True)
        text = part
    print()
    return text

def _build_skill_meta(mgr):
    """Gather unique metadata for every skill in all executor cards."""
    unique_skills = {}  # Use a dictionary to store skills by their ID
    for _, card, _, _ in mgr.skills.values():
        for s in card.skills:
            if s.id not in unique_skills:
                unique_skills[s.id] = {
                    "skill_id": s.id,
                    "name": s.name,
                    "description": getattr(s, "description", None),
                    "tags": getattr(s, "tags", []),
                    "examples": getattr(s, "examples", None),

                }
    return list(unique_skills.values()) # Convert the dictionary values back to a list


async def _send_task(mgr, client, card, session, question, push=False, host=None, port=None) -> str:
    """Build a card-driven payload (with optional push) and dispatch it."""
    # Input parts
    content = {"question": question}
    modes   = getattr(card, "acceptedInputModes", ["text"])
    parts   = ([{"type": "json", "json": content}]
               if "json" in modes
               else [{"type": "text", "text": json.dumps(content)}])

    # Optional push URL & auth
    can_push = push and getattr(card.capabilities, "pushNotifications", False)
    push_url = (urllib.parse.urljoin(f"http://{host}:{port}", "/notify")
                if can_push and host and port else None)
    schemes = getattr(card.authentication, "supportedSchemes", ["bearer"])

    # Assemble payload
    payload = {
        "id": uuid4().hex,
        "sessionId": session,
        "acceptedOutputModes": card.defaultOutputModes,
        "message": {"role": "user", "parts": parts},
        **({"pushNotification": {"url": push_url,
                                 "authentication": {"schemes": schemes}}}
           if push_url else {})
    }

    # Dispatch, letting the card decide streaming vs one-shot
    stream = getattr(card.capabilities, "streaming", False)
    return await _send_payload(client, card, session, payload, stream)

#### 4.3. Orchestration Logic

The `orchestrate` function coordinates the multi-agent interaction:

1. **Planning Phase**: It queries the Planner Agent (via A2A) with the user's question and metadata about available skills from other agents. The Planner Agent returns a JSON plan (a list of `skill_id`s).

2. **Execution Phase**: It iterates through the plan, calling the appropriate Skill Executor A2A Agents (e.g., Custom Tool Agent's skills) for each step.

3. **Composition Phase**: Finally, it sends the original question and the collected results from the execution phase to the Composer A2A Agent to generate a polished, human-readable response.

In [14]:
async def orchestrate(
    agent_manager: AgentManager,
    question: str,
    push: bool = False,
    push_receiver: str = "http://localhost:5000",
) -> str:
    # Unpack orchestrator info
    orch_url, orch_card, orch_client, orch_session = agent_manager.orchestrator

    # Optionally start push listener
    host = port = None
    if push:
        parsed = urllib.parse.urlparse(push_receiver)
        host, port = parsed.hostname, parsed.port
        auth = PushNotificationReceiverAuth()
        await auth.load_jwks(f"{orch_url}/.well-known/jwks.json")
        PushNotificationListener(host, port, auth).start()

    # --- Planning Phase ---
    print("\n\033[1;33m=========== 🧠 Planning Phase ===========\033[0m")
    # Build skill metadata
    skills_meta = _build_skill_meta(agent_manager)
    plan_instructions = (
        "You are an orchestration assistant.\n"
        "Available skills (id & name & description & tags & examples):\n"
        f"{json.dumps(skills_meta, indent=2)}\n\n"
        "When given a user question, respond _only_ with a JSON array of objects, "
        "each with key `skill_id`, without any surrounding object. You may be asked to write single or multiple skills.\n"
        "For example for multiple tools:\n"
        "["
        "{\"skill_id\": \"tool_1\"}, "
        "{\"skill_id\": \"tool_2\"}"
        "]"
    )
    combined = plan_instructions + "\n\nUser question: " + question
    raw = await _send_task(agent_manager, orch_client, orch_card, orch_session, combined, push=push, host=host, port=port)
    print(f"Raw plan ➡️ {raw}")
    try:
        plan = json.loads(raw[: raw.rfind("]") + 1])
    except ValueError:
        print("\033[31mPlan parse failed, fixing invalid JSON...\033[0m")
        fixer = "Fix this json to be valid: " + raw
        fixed = await _send_task(agent_manager, orch_client, orch_card, orch_session, fixer, push=push, host=host, port=port)
        plan = json.loads(fixed)
    print(f"\n\033[1;32mFinal plan ➡️ {plan}\033[0m")

    # --- Execution Phase ---
    print("\n\033[1;33m=========== ⚡️ Execution Phase ===========\033[0m")
    parts = []
    for i, step in enumerate(plan, 1):
        sid = step["skill_id"]
        inp = json.dumps(step.get("input", {}))
        print(f"➡️ Step {i}: {sid}({inp})")

        info = agent_manager.skills.get(sid)
        if not info:
            print(f"\033[31mNo executor for '{sid}', skipping.\033[0m")
            parts.append({"skill_id": sid, "output": None})
            continue

        _, skill_card, skill_client, skill_sess = info
        out = await _send_task(agent_manager, skill_client, skill_card, skill_sess, f"{sid}({inp})", push=push, host=host, port=port)
        print(f"   ✅ → {out}")
        parts.append({"skill_id": sid, "output": out})

    # --- Composing Answer ---
    print("\n\033[1;33m=========== 🛠️ Composing Answer ===========\033[0m")
    comp_prompt = (
        f"Using the following information: {json.dumps(parts)}, "
        f"write a clear and human-friendly response to the question: '{question}'. "
        "Keep it concise and easy to understand and respond like a human with character. "
        "Only use the information provided. If you cannot answer the question, say 'I don't know'. "
        "Never show any code or JSON or Markdown, just the answer.\n\n"
    )
    _, write_card, write_client, write_sess = agent_manager.skills["writing_agent"]
    final = await _send_task(agent_manager, write_client, write_card, write_sess, comp_prompt, push=push, host=host, port=port)

    print("\n\033[1;36m🎉 FINAL ANSWER\033[0m")
    print(final)
    print("\033[1;36m====================================\033[0m")
    return final

### 5. Running the Orchestration
Now we define the URLs for our orchestrator (Planner Agent) and skill executor agents (Custom Tool Agent, Composer Agent). We then initialize the `AgentManager` and call the `orchestrate` function with sample questions.

The `AgentManager` uses the first URL for the orchestrator/planner and the rest for skill executors. The `orchestrate` function will then:

1. Query the Planner Agent with the user's question and the list of available skills.

2. The Planner Agent returns a plan (e.g. `[{'skill_id': 'random_number_tool'}]`).

3. The `orchestrate` function executes the plan by calling the specified skill agents.

4. Finally, it sends the original question and the skill outputs to the Composer Agent to generate a polished, human-readable response.

In [15]:
ORCHESTRATOR_URL = "http://localhost:10020"
EXECUTOR_URLS    = ["http://localhost:10021", "http://localhost:10022"]
URLS             = [ORCHESTRATOR_URL, *EXECUTOR_URLS]

_agent_manager = AgentManager(URLS)
orch_url, orch_card, *_ = _agent_manager.orchestrator

print("\n\033[1;36m===================== 🛰️ Connected Agents =====================\033[0m")
print(f"Orchestrator: {orch_url} ({orch_card.name})")
print("Executors:")
for sid, (u, card, *_) in _agent_manager.skills.items():
    print(f"  • {sid} -> {u} ({card.name})")
print("\033[1;36m===============================================================\033[0m")

questions = [ 
    "Get todays date then generate five random numbers",
    "Get todays date?",
    "I want one random number",
    ]

for question in questions:
    await orchestrate(_agent_manager, question)


===================== 🛰️ Connected Agents =====================
Orchestrator: http://localhost:10020 (Orchestration Agent)
Executors:
  • random_number_tool -> http://localhost:10021 (Custom Agent)
  • date_tool -> http://localhost:10021 (Custom Agent)
  • writing_agent -> http://localhost:10022 (Writing Agent)

=========== 🧠 Planning Phase ===========
Raw plan ➡️ [
  {"skill_id": "date_tool"},
  {"skill_id": "random_number_tool"},
  {"skill_id": "random_number_tool"},
  {"skill_id": "random_number_tool"},
  {"skill_id": "random_number_tool"},
  {"skill_id": "random_number_tool"}
]

Final plan ➡️ [{'skill_id': 'date_tool'}, {'skill_id': 'random_number_tool'}, {'skill_id': 'random_number_tool'}, {'skill_id': 'random_number_tool'}, {'skill_id': 'random_number_tool'}, {'skill_id': 'random_number_tool'}]

=========== ⚡️ Execution Phase ===========
➡️ Step 1: date_tool({})
   ✅ → {
    "type": "function",
    "name": "date_tool",
    "parameters": {}
}Tool:date_tool Args:{}Tool:date_tool R

## 6. Wrapping Up & Future Directions

We've successfully orchestrated a team of specialized Llama Stack agents, showcasing how they can collaborate via the A2A protocol to tackle complex queries.

**What We Achieved:**

* We configured the Llama Stack environment and designed three distinct agents: a `Planner`, a `Custom Tool Agent` (with date/random number skills), and a `Composer`.

* Each agent was made accessible through A2A, with an `AgentManager` and an `orchestrate` function guiding their interaction to deliver user-friendly answers.

The key insight is the power of modular, specialized agents communicating through a standard protocol, allowing for flexible and scalable AI system development.

### Future Directions:

Inspired? Here are a few ways to build on this foundation:

* **Refine & Expand**: Experiment with agent instructions or add new tools and specialized agents to the team.

* **Boost Orchestration**: Explore more dynamic planning, such as conditional logic or inter-agent feedback loops.

* **Challenge the System**: Test with increasingly complex queries to push the boundaries of the current setup.

This notebook serves as a stepping stone into the exciting world of multi-agent AI. We hope it empowers you to build even more sophisticated applications. Happy coding!